In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

# ==========================================
# 1. LOAD THE RAW DATA
# ==========================================
print("🔄 Loading Raw Data...")

# Load Airbnb Listings
df_airbnb = pd.read_csv('../data/raw/airbnb_listings.csv.gz', compression='gzip', low_memory=False)

# Load NYC Zip Code Map (GeoJSON)
gdf_nyc = gpd.read_file('../data/raw/nyc_zip_codes.geojson')

# Load Zillow Home Values
df_zillow = pd.read_csv('../data/raw/zillow_home_values.csv')

print(f"   - Airbnb Rows: {df_airbnb.shape[0]}")
print(f"   - NYC Zip Codes: {gdf_nyc.shape[0]}")
print("✅ Data Loaded.")

# ==========================================
# 2. CLEAN AIRBNB DATA
# ==========================================
print("\n🧹 Cleaning Airbnb Data...")

# Filter: Investors only want 'Entire home/apt'
df_airbnb = df_airbnb[df_airbnb['room_type'] == 'Entire home/apt'].copy()

# Clean Price: Remove '$', ',', and convert to Float
df_airbnb['price'] = df_airbnb['price'].astype(str).str.replace('$', '').str.replace(',', '').astype(float)

# Filter: Remove outliers (e.g., $0 or > $2000/night)
df_airbnb = df_airbnb[(df_airbnb['price'] > 50) & (df_airbnb['price'] < 2000)]

# Select only useful columns
cols_to_keep = ['id', 'listing_url', 'name', 'host_id', 'latitude', 'longitude', 
                'room_type', 'price', 'minimum_nights', 'number_of_reviews', 
                'reviews_per_month', 'availability_365', 'accommodates', 
                'bedrooms', 'beds', 'bathrooms_text']

# Keep only columns that actually exist in the file (safeguard)
existing_cols = [c for c in cols_to_keep if c in df_airbnb.columns]
df_airbnb = df_airbnb[existing_cols]

print(f"   - Rows after cleaning: {df_airbnb.shape[0]}")

# ==========================================
# 3. THE SPATIAL JOIN (The Magic Step 🪄)
# ==========================================
print("\n🗺️ Performing Spatial Join (Mapping Lat/Lon to Zip Codes)...")

# Convert Airbnb Data to a GeoDataFrame (Using Lat/Lon)
gdf_airbnb = gpd.GeoDataFrame(
    df_airbnb, 
    geometry=gpd.points_from_xy(df_airbnb.longitude, df_airbnb.latitude),
    crs="EPSG:4326" # Standard GPS Coordinates
)

# Ensure NYC Map uses the same coordinate system
if gdf_nyc.crs != "EPSG:4326":
    gdf_nyc = gdf_nyc.to_crs("EPSG:4326")

# Perform the Join: "Which Zip Code polygon contains this Airbnb point?"
# We use 'inner' join to keep only listings that fall INSIDE a valid NYC Zip
gdf_joined = gpd.sjoin(gdf_airbnb, gdf_nyc, how="inner", predicate="within")

# Rename the map column (usually 'MODZCTA' or 'postalCode') to 'zipcode'
# Note: Check your GeoJSON column names if this fails. NYC OpenData usually uses 'modzcta'
if 'modzcta' in gdf_joined.columns:
    gdf_joined = gdf_joined.rename(columns={'modzcta': 'zipcode'})
elif 'postalCode' in gdf_joined.columns:
    gdf_joined = gdf_joined.rename(columns={'postalCode': 'zipcode'})

print(f"   - Listings successfully mapped to a Zip Code: {gdf_joined.shape[0]}")

# ==========================================
# 4. PREPARE ZILLOW DATA
# ==========================================
print("\n🏠 Preparing Zillow Cost Data...")

# Zillow data is 'Wide' (Dates are columns). We need the most recent column.
# Filter for NYC only (The Zillow file covers the whole US)
df_zillow['RegionName'] = df_zillow['RegionName'].astype(str) # Ensure Zip is text
nyc_zips = gdf_joined['zipcode'].unique().astype(str) # Get Zips from our map
df_zillow_nyc = df_zillow[df_zillow['RegionName'].isin(nyc_zips)].copy()

# Get the last column (the most recent price)
latest_price_col = df_zillow_nyc.columns[-1] 
print(f"   - Using Zillow Price Data from: {latest_price_col}")

# Create a clean reference table
df_cost = df_zillow_nyc[['RegionName', latest_price_col]].copy()
df_cost.columns = ['zipcode', 'zillow_price']

# ==========================================
# 5. FINAL MERGE & EXPORT
# ==========================================
print("\n🔗 Merging Revenue (Airbnb) & Cost (Zillow)...")

# Ensure zipcodes are strings in both tables
gdf_joined['zipcode'] = gdf_joined['zipcode'].astype(str)
df_cost['zipcode'] = df_cost['zipcode'].astype(str)

# Merge
df_master = pd.merge(gdf_joined, df_cost, on='zipcode', how='inner')

# Drop geometry to save as CSV (we don't need the point shapes anymore)
df_master = pd.DataFrame(df_master.drop(columns='geometry'))

# Save
output_path = '../data/processed/nyc_rental_arbitrage_master.csv'
df_master.to_csv(output_path, index=False)

print(f"✅ SUCCESS! Master Table saved to: {output_path}")
print(f"   - Final Dataset Size: {df_master.shape[0]} rows")
print("   - Sample Columns:", df_master.columns.tolist())

🔄 Loading Raw Data...
   - Airbnb Rows: 36111
   - NYC Zip Codes: 178
✅ Data Loaded.

🧹 Cleaning Airbnb Data...
   - Rows after cleaning: 11892

🗺️ Performing Spatial Join (Mapping Lat/Lon to Zip Codes)...
   - Listings successfully mapped to a Zip Code: 11891

🏠 Preparing Zillow Cost Data...
   - Using Zillow Price Data from: 2025-10-31

🔗 Merging Revenue (Airbnb) & Cost (Zillow)...
✅ SUCCESS! Master Table saved to: ../data/processed/nyc_rental_arbitrage_master.csv
   - Final Dataset Size: 11744 rows
   - Sample Columns: ['id', 'listing_url', 'name', 'host_id', 'latitude', 'longitude', 'room_type', 'price', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'availability_365', 'accommodates', 'bedrooms', 'beds', 'bathrooms_text', 'index_right', ':id', ':version', ':created_at', ':updated_at', 'zipcode', 'label', 'zcta', 'pop_est', 'zillow_price']
